In [1]:
import os
import sys
home_dir = "../"
sys.path.append(home_dir)

import pandas as pd
import numpy as np
from joblib import delayed, Parallel
from typing import List
# from torchmetrics.functional import pairwise_cosine_similarity
import remote_homologs.pickle_utils as pickle_utils
import csv
from sklearn.metrics import precision_recall_curve, auc, roc_curve

In [7]:
scop_df = pd.read_csv('../data/SCOP/processed/SCOP_with_seq.tsv',sep='\t')

In [12]:
scop_df

,FA-DOMID,FA-PDBID,FA-PDBREG,FA-UNIID,FA-UNIREG,SF-DOMID,SF-PDBID,SF-PDBREG,SF-UNIID,SF-UNIREG,SCOPCLA,TP,CL,CF,SF,FA,seq
0,8045703,3H8D,C:1143-1264,Q64331,1143-1264,8091604,3H8D,C:1143-1264,Q64331,1143-1264,"TP=1,CL=1000003,CF=2001470,SF=3002524,FA=4004627",1,1000003,2001470,3002524,4004627,DMKRQQRFFRIPFIRPADQYKDPQNKKKGWWYAHFDGPWIARQMEL...
1,8094330,6J56,A:1158-1282,Q9UM54,1167-1291,8094331,6J56,A:1158-1282,Q9UM54,1167-1291,"TP=1,CL=1000003,CF=2001470,SF=3002524,FA=4004627",1,1000003,2001470,3002524,4004627,RQREIEMNRQQRFFRIPFIRPADQYKDPQSKKKGWWYAHFDGPWIA...
2,8017835,3FKQ,A:1-116,D0VX10,1-116,8017836,3FKQ,A:1-116,D0VX10,1-116,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4003986",1,1000002,2000016,3001156,4003986,MKIKVALLDKDKEYLDRLTGVFNTKYADKLEVYSFTDEKNAIESVK...
3,8021315,1XHF,A:2-122,P0A9Q1,2-122,8033695,1XHF,A:2-122,P0A9Q1,2-122,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4003632",1,1000002,2000016,3001156,4003632,QTPHILIVEDELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEYD...
4,8021787,1Y7P,B:79-215,O28869,79-215,8034167,1Y7P,B:79-215,O28869,79-215,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4002481",1,1000002,2000016,3001156,4002481,SFERVFGKRVIILGGGALVSQVAIGAISEADRHNLRGERISVDTMP...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31071,8022870,1SCJ,B:307-377,P04189,36-106,8035250,1SCJ,B:307-377,P04189,36-106,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001246",1,1000003,2000014,3001155,4001246,EKKYIVGFKQTMSAMSSAKKKDVISEKGGKVQKQFKYVNAAAATLD...
31072,8027500,1T1E,A:12-188,Q8RR56,12-188,8039879,1T1E,A:12-188,Q8RR56,12-188,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001246",1,1000003,2000014,3001155,4001246,EKREVLAGHARRQAPQAVDKGPVTGDQRISVTVVLRRQRGDELEAH...
31073,8027593,1JQG,A:4P-100P,O97389,18-109,8039972,1JQG,A:4P-100P,O97389,18-109,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001250",1,1000003,2000014,3001155,4001250,HEIYDGHAVYQVDVASMDQVKLVHDFENDLMLDVWSDAVPGRPGKV...
31074,8028119,1KN6,A:4-76,P63239,31-103,8040498,1KN6,A:4-76,P63239,31-103,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4002630",1,1000003,2000014,3001155,4002630,FVNEWAAEIPGGQEAASAIAEELGYDLLGQIGSLENHYLFKHKSHP...


In [21]:
scop_df['SF-UNIID'][scop_df['SF-UNIID'].apply(lambda x: '1jlz' in x.lower())]

15972    A0A2K1JLZ3
Name: SF-UNIID, dtype: object

In [100]:
small_part = True

In [2]:
from typing import Any, Mapping, Optional, Sequence


In [24]:
data = pd.read_pickle('../data/SCOPe/contrib_hhblits/scope_th95_hhsearch_logeval_dct.pk')

In [4]:
list( data[list(data.keys())[0]].values())[-10:]


['-15.093',
 '-15.094',
 '-15.095',
 '-15.096',
 '-15.098',
 '-15.105',
 '-15.108',
 '-15.110',
 '-15.113',
 '-15.115']

In [5]:
np.matrix(pd.Series([np.array([1.1,2.2,3.2]),np.array([1.1,2.2,3.2])]).tolist())

matrix([[1.1, 2.2, 3.2],
        [1.1, 2.2, 3.2]])

In [11]:
# # [line for line in lines if 'NAME' in line]

# all_seqs = pd.read_csv('~/PROJECTS/remote_homologs/data/SCOPe/contrib_hhblits/scope_th95.fas', sep='\t', lineterminator='>', header=None)

# len(all_seqs[0].apply(lambda x: x.split()[0]).unique())

# all_seqs

# all_seqs[0]=all_seqs[0].apply(lambda x: '>'+ x)

# chunks = np.array_split(all_seqs, 10)

# chunks

# for i,chunk in enumerate(chunks):
#     chunk.to_csv('~/PROJECTS/remote_homologs/data/SCOPe/contrib_hhblits/scope_th95_chunk'+str(i)+'.fas', header=False, index=False, quoting=csv.QUOTE_NONE, escapechar=' ')

In [12]:
def compute_remote_homolog_labels(remote_homologs_df: pd.DataFrame, pos_col:str, neg_col:str):
    # pos_col=superfamily_col, neg_col=family_col
    # pos_col=fold_col, neg_col=superfamily_col

    n_data_points = remote_homologs_df.shape[0]

    def compute_labels(i):
        labels = np.zeros((n_data_points), dtype=int) # 0 is the negative label
        for j in range(n_data_points):
            if remote_homologs_df.loc[i, neg_col] == remote_homologs_df.loc[j, neg_col]:
                labels[j] = -1 # will not be considered in the metric computation
            elif remote_homologs_df.loc[i, pos_col] == remote_homologs_df.loc[j, pos_col]:
                labels[j] = 1 # positive label
        return labels


        # for j in range(n_data_points):
        #     if (remote_homologs_df.loc[i, pos_col] == remote_homologs_df.loc[j, pos_col]) and (remote_homologs_df.loc[i, neg_col] != remote_homologs_df.loc[j, neg_col]):
        #         labels[j] = 1 # positive label
        # return labels


    with Parallel(n_jobs=os.cpu_count(), verbose=1) as parallel:
        list_of_labels = parallel(delayed(compute_labels)(i)
            for i in range(remote_homologs_df.shape[0]))

    labels = np.stack(list_of_labels)
    print(f"log: finished computing ground-truth labels {labels.shape}")
    return labels


In [1]:
import numpy as np

In [2]:
def compute_auroc(y_true:np.array, y_pred:np.array):
    fpr, tpr, ths = roc_curve(y_true, y_pred)
    return auc(fpr, tpr)

def compute_auprc(y_true:np.array, y_pred:np.array):
    precision, recall, ths = precision_recall_curve(y_true, y_pred)
    return auc(recall, precision)

def compute_hitk(y_true:np.array, y_pred:np.array, k=10):
    """
    y_true (numpy.ndarray): Array of actual values (1 for relevant items, 0 for irrelevant items).
    y_pred (numpy.ndarray): Array of predicted values (1 for recommended items, 0 for non-recommended items).
    """
    top_k_indices = np.argsort(y_pred)[::-1][:k]
    hits = any(y_true[i] == 1 for i in top_k_indices)
    hit_at_k = 1 if hits else 0
    return hit_at_k

# y_true, y_pred = np.array([1, 0, 1, 0, 1]), np.array([.7, .2, .8, .3, .9])
# compute_hitk(y_true, y_pred, k=1)

In [15]:
compute_hitk([1,0,0,0,0,1],[0.98,0,0,0.99,0.99,0.97],6)

1

In [14]:
def get_data_specific_parameters(data_name):
    # SCOP parameters
    if data_name == "SCOP":
        sep = "\t"
        seq_id_col, family_col, superfamily_col, fold_col = "FA-DOMID", "FA", "SF", "CF"
        data_filepath = home_dir + f"data/{data_name}/processed/SCOP_with_seq.tsv"

    # SCOPe parameters
    elif data_name == "SCOPe":
        sep = ","
        seq_id_col, family_col, superfamily_col, fold_col = "sid", "family", "superfamily", "fold"
        data_filepath = home_dir + f"data/{data_name}/all_sequences.csv"

    return sep, seq_id_col, family_col, superfamily_col, fold_col, data_filepath

In [15]:
included_hhblits_seqs = list(data.keys())

## The main scripts starts from here

In [16]:
# if __name__ == "__main__":
data_name = "SCOPe" # SCOP, SCOPe
remote_homology_level = "superfamily" # superfamily, fold
model_name = "hhblits"
# random, tapebert, proteinbert, esm1b_t33_650M_UR50S, esm2_t33_650M_UR50D, prottrans_bert_bfd, prottrans_albert_bfd, prottrans_t5_bfd

In [17]:
sep, seq_id_col, family_col, superfamily_col, fold_col, data_filepath = get_data_specific_parameters(data_name)
# embeddings_dirpath = home_dir+f"data/{data_name}/embeddings/{model_name}/"
ths = [10, 20, 30, 40, 70, 95]

In [18]:
# setting up positive and negative col label based on remote_homology_level
# superfamily level remote homologs
if remote_homology_level == "superfamily":
    pos_col, neg_col = superfamily_col, family_col
# fold level remote homology
elif remote_homology_level == "fold":
    pos_col, neg_col = fold_col, superfamily_col
else:
    raise f"remote_homology_level={remote_homology_level} not refined. Option: superfamily, fold"

In [20]:
# import pickle 

# scores_dir = '../data/SCOPe/contrib_hhblits/'
# fns = ['hhblits_all_th95_query_fast_chunk'+str(i)+'.scores.ffdata' for i in range(10)]

# data = {}
# current_entry = {}
# i=0

In [ ]:
# for fn in fns:
#     scores_path = scores_dir + fn
#     with open(scores_path, 'r') as file:
#         for line in file:
#             i+=1
#             # if i>60000:
#             #     break
#             line = line.strip().replace('\x00','')
#             if line.startswith("NAME"):
#                 if current_entry:
#                     data[current_entry['FILE']] = current_entry
#                     current_entry = {}
#             elif line:
#                 key, value = line.split(maxsplit=1)
#                 try:
#                     if key !='FILE':
#                         value = value.split()[8]
#                 except:
#                     pass
#                 current_entry[key] = value
#     break

# list(data.keys())[0]

# len(list(data.keys()))

# sids = [pd.read_csv((scores_dir+'scope_th95_chunk'+str(i)+'.fas.ffindex'),sep='\t', names=['sid','a','b'])['sid'] for i in range(10)]

# len(pd.concat(sids))

# for s in ['ffindex_from_fasta -s scope_th95_chunk'+str(i)+'.fas.ff{data,index} scope_th95_chunk'+str(i)+'.fas' for i in range(10)]:
    # print(s)

# included_hhblits_seqs[:10]

# '../data/SCOPe/contrib_hhblits/'
# fns = ['hhblits_all_th95_query_fast_chunk'+str(i)+'.scores.ffdata' for i in range(10)]

In [33]:
# loading data and creating main dicts
data_df = pd.read_csv(data_filepath, sep=sep)

In [101]:
if small_part:
    data_df = data_df.head(100)

In [102]:
# data_df = data_df[data_df['sid'].apply(lambda x: x in included_hhblits_seqs)]

In [103]:
# data_df

In [104]:
int2sid_dict = data_df[seq_id_col].to_dict()
sid2int_dict = {sid: i for i, sid in int2sid_dict.items()}

In [105]:
def get_similarities_for_sid(sid):
    sims=[]
    if sid not in data.keys():
        return np.array([float(-1)]*len(data_df),dtype=float)
    for sid2 in data_df['sid']:
        if sid2 in data[sid].keys():
            sims.append(float(data[sid][sid2]))
        else:
            sims.append(float(0))
    return np.array(sims,dtype=float)

In [167]:
similarities = data_df['sid'].apply(get_similarities_for_sid)

In [168]:
# similarities = similarities.apply(lambda x: [float(n) for n in x])

In [169]:
similarities = np.array(similarities.tolist())

In [170]:
similarities

array([[ 77.846,   0.   ,   0.   , ...,   0.   ,   0.   ,  -7.207],
       [ -1.   ,  -1.   ,  -1.   , ...,  -1.   ,  -1.   ,  -1.   ],
       [ -1.   ,  -1.   ,  -1.   , ...,  -1.   ,  -1.   ,  -1.   ],
       ...,
       [ -1.   ,  -1.   ,  -1.   , ...,  -1.   ,  -1.   ,  -1.   ],
       [ -1.   ,  -1.   ,  -1.   , ...,  -1.   ,  -1.   ,  -1.   ],
       [-10.999,   0.   ,   0.   , ...,   0.   ,   0.   ,  54.084]])

In [171]:
# len([x for x in similarities if x[0]!=-1])

In [172]:
# # for debugging purpose onlu
# # checking if the any embeddings contain nan values, and removing those file. 
# # then the data generator script needs to run again
# x = []
# for i, sid in enumerate(data_df[seq_id_col].to_list()):
#     rep = pickle_utils.load(embeddings_dirpath + str(sid) + ".pkl")
#     # print(rep.shape)
#     if np.isnan(rep).any() == True:
#         print(i, sid, f"removing {embeddings_dirpath + str(sid) + '.pkl'}")
#         # os.remove(embeddings_dirpath + str(sid) + ".pkl") # deleting this file
#         x.append(i)
#     # break
# print(len(x))
# x

In [173]:
# # computing embedding similarities
# if model_name=="random":
#     similarities = compute_embeddings_similarities(None, None, n_embeddings_to_load=data_df.shape[0], is_random=True)
# else:
#     similarities = compute_embeddings_similarities(data_df[seq_id_col].to_list(), embeddings_dirpath, data_df.shape[0], is_random=False)

In [174]:
model_name

'hhblits'

In [175]:
# for th in ths:
th=10
# creating output directory and files
out_dir = home_dir+f"data/{data_name}/ranking_results/{model_name}/{remote_homology_level}/th_{th}/"
os.makedirs(out_dir, exist_ok=True)
final_results_per_queries_filepath = f"{out_dir}results_per_query.tsv"
final_results_per_remhom_filepath = f"{out_dir}results_per_{remote_homology_level}.tsv"
weighted_results_filepath = f"{out_dir}weighted_results.tsv"
non_weighted_results_filepath = f"{out_dir}non_weighted_results.tsv"

# checking whether the outputs are already computed
if os.path.exists(final_results_per_queries_filepath) and os.path.exists(final_results_per_remhom_filepath) and os.path.exists(weighted_results_filepath) and os.path.exists(non_weighted_results_filepath):
    # continue
    print("results exists")

results exists


In [176]:
th

10

In [177]:
# loading data at th
remote_homologs_filepath = home_dir+f"data/{data_name}/processed_at_th/th_{str(th)}.tsv"
remote_homologs_df = pd.read_csv(remote_homologs_filepath, sep="\t")
remhom_int2sid_dict = remote_homologs_df[seq_id_col].to_dict()
remhom_sid2int_dict = {sid: i for i, sid in remhom_int2sid_dict.items()}

In [ ]:
# computing ground-truth labels
labels = compute_remote_homolog_labels(remote_homologs_df, pos_col=pos_col, neg_col=neg_col)

[Parallel(n_jobs=64)]: Using backend LokyBackend with 64 concurrent workers.
Exception ignored in: <function _releaseLock at 0x7fc74b253dc0>
Traceback (most recent call last):
  File "/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Exception ignored in: <function _releaseLock at 0x7fc74b253dc0>
Traceback (most recent call last):
  File "/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Exception ignored in: <function _releaseLock at 0x7fc74b253dc0>
Traceback (most recent call last):
  File "/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Exception ignored in: Traceback (most recent call last):
  File "/opt/sw/spack/a

In [186]:
labels.shape

(6784, 6784)

In [179]:
# filtering out the similarities and lables corresponding to each other
similarity_indices_to_keep =[sid2int_dict[sid] for sid in remote_homologs_df[seq_id_col] if sid in sid2int_dict]
print(len(similarity_indices_to_keep))

label_indices_to_keep = [remote_homologs_df[remote_homologs_df[seq_id_col]==int2sid_dict[idx]].index[0] for idx in similarity_indices_to_keep]
print(len(label_indices_to_keep))

labels_specific_to_conditions = labels[label_indices_to_keep, :]
labels_specific_to_conditions = labels_specific_to_conditions[:, label_indices_to_keep]
print(labels_specific_to_conditions.shape)

ERROR! Session/line number was not unique in database. History logging moved to new session 953
6
6
(6, 6)


In [188]:
similarities[similarity_indices_to_keep]

array([[ 71.964,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,  70.62 ,   0.   ,
          0.   ,   0.   ,  69.655,   0.   ,  65.981,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,  68.926,  79.624,  74.638,  -9.607,
         -2.858,   0.   ,   0.   ,   0.   ,   0.   ,  -6.529,  -8.841,
         -9.11 ,  -8.063,   0.   ,   0.   ,   0.   ,  -9.58 ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         -8.245,  -6.73 ,  -7.28 ,  -8.861,  -9.123,  -0.865,  -8.753,
         -4.769,  -7.145,  -8.334,  -1.907,   0.   ,   0.   ,   0.   ,
         -7.51 ,  -4.393,   0.   ,   0.   ,  -6.93 ,  -5.677,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,  -7.2  ,   0.   ,   0.   ,
         -4.843,  -7.502,  -4.501,   0.   ,  -7.112,  -6.956,  -4.371,
          0.   ,   0.   ,   0.   ,  -6.077,  -5.261,   0.   ,   0.   ,
      

In [180]:
similarities[similarity_indices_to_keep, :][:, similarity_indices_to_keep]

array([[ 79.624,  -9.607,  -9.58 ,  -8.753,  -1.907,  -4.393],
       [ -7.592,  65.31 ,  41.326,  28.166, -11.911,  25.203],
       [ -9.829,  25.907,  48.099,  15.296, -12.148,  14.288],
       [ -4.648,  41.151,  37.33 ,  61.967,  -7.03 ,  39.766],
       [  1.898, -10.792,  -8.355,  -5.119,  81.385,  -8.433],
       [ -8.413,  44.476,  35.518,  33.481, -12.837,  52.838]])

In [181]:
similarities_specific_to_conditions

array([[ 79.624,  -9.607,  -9.58 ,  -8.753,  -1.907,  -4.393],
       [ -7.592,  65.31 ,  41.326,  28.166, -11.911,  25.203],
       [ -9.829,  25.907,  48.099,  15.296, -12.148,  14.288],
       [ -4.648,  41.151,  37.33 ,  61.967,  -7.03 ,  39.766],
       [  1.898, -10.792,  -8.355,  -5.119,  81.385,  -8.433],
       [ -8.413,  44.476,  35.518,  33.481, -12.837,  52.838]])

In [182]:
similarities_specific_to_conditions = similarities[similarity_indices_to_keep, :]
similarities_specific_to_conditions = similarities_specific_to_conditions[:, similarity_indices_to_keep]
print(similarities_specific_to_conditions.shape)

assert len(similarity_indices_to_keep)==len(label_indices_to_keep), "number of labels and similarities mismatch"

(6, 6)


In [183]:
# computing results per query
n_queries = labels_specific_to_conditions.shape[0]
results = []
for query_id in range(n_queries):
    # query_id = 50
    mask = labels_specific_to_conditions[query_id] != -1

    n_true_pos = (labels_specific_to_conditions[query_id]==1).sum()
    n_true_neg = (labels_specific_to_conditions[query_id]==0).sum()
    y_true, y_pred = labels_specific_to_conditions[query_id][mask], similarities_specific_to_conditions[query_id][mask]

    assert mask.sum()==labels_specific_to_conditions[query_id][mask].shape[0]==similarities_specific_to_conditions[query_id][mask].shape[0], f"mask did not match for {query_id: sid2int_dict[query_id]}"

    auroc = compute_auroc(y_true, y_pred) # same as roc_auc_score(y_true, y_pred)
    auprc = compute_auprc(y_true, y_pred)
    hit1 = compute_hitk(y_true, y_pred, k=1)
    hit10 = compute_hitk(y_true, y_pred, k=10)
    # print(auroc, auprc, hit10, n_true_pos, n_true_neg)

    data_point = dict(sid=remhom_int2sid_dict[query_id], auroc=auroc, auprc=auprc, hit1=hit1, hit10=hit10, n_true_pos=n_true_pos, n_true_neg=n_true_neg)
    results.append(data_point)

    # break
    # if query_id==50: break

    if query_id%1000==0: 
        print(f"processed: {query_id}")

results_df = pd.DataFrame.from_dict(results)

processed: 0


/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1007: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1007: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1007: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1007: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/opt/sw/spack/apps/l

In [184]:
similarities_specific_to_conditions[query_id][mask]

array([-8.413])

In [153]:
np.array(similarities_specific_to_conditions)[query_id]

array([ -8.413,  44.476,  35.518,  33.481, -12.837,  52.838])

In [154]:
int2sid_dict[0]

'd2gkma_'

In [155]:
data_df

,Unnamed: 0,sid,sccs,span,sequence,SF,fold,sequence_len,remote_homologs,SF_count
0,0,d2gkma_,a.1.1.1,(A:),gllsrlrkrepisiydkiggheaievvvedffvrvladdqlsaffs...,a.1.1,a.1,127,"['d1asha_', 'd1vhba_', 'd4vhba_', 'd2vhba_', '...",714
1,1,d2gkmb_,a.1.1.1,(B:),gllsrlrkrepisiydkiggheaievvvedffvrvladdqlsaffs...,a.1.1,a.1,126,"['d1asha_', 'd1vhba_', 'd4vhba_', 'd2vhba_', '...",714
2,2,d2gl3a_,a.1.1.1,(A:),gllsrlrkrepisiydkiggheaievvvedffvrvladdqlsaffs...,a.1.1,a.1,128,"['d1asha_', 'd1vhba_', 'd4vhba_', 'd2vhba_', '...",714
3,3,d2gl3b_,a.1.1.1,(B:),gllsrlrkrepisiydkiggheaievvvedffvrvladdqlsaffs...,a.1.1,a.1,127,"['d1asha_', 'd1vhba_', 'd4vhba_', 'd2vhba_', '...",714
4,4,d1idra_,a.1.1.1,(A:),gllsrlrkrepisiydkiggheaievvvedfyvrvladdqlsaffs...,a.1.1,a.1,127,"['d1asha_', 'd1vhba_', 'd4vhba_', 'd2vhba_', '...",714
...,...,...,...,...,...,...,...,...,...,...
95,95,d3n48a_,a.1.1.2,(A:),vlspadktnikstwdkigghagdyggealdrtfqsfpttktyfphf...,a.1.1,a.1,141,"['d2gkma_', 'd2gkmb_', 'd2gl3a_', 'd2gl3b_', '...",714
96,96,d1s0ha1,a.1.1.2,(A:1-141),vlsaadktnvkaawskvggnagefgaealermflgfpttktyfphf...,a.1.1,a.1,141,"['d2gkma_', 'd2gkmb_', 'd2gl3a_', 'd2gl3b_', '...",714
97,97,d2b7hb1,a.1.1.2,(B:2-145),hltaeekslvsglwakvnvdevggealgrllivypwtqrffdsfgd...,a.1.1,a.1,144,"['d2gkma_', 'd2gkmb_', 'd2gl3a_', 'd2gl3b_', '...",714
98,98,d2h8fa_,a.1.1.2,(A:),slsdkdkaavralwskigksadaigndalsrmivvypqtktyfshw...,a.1.1,a.1,142,"['d2gkma_', 'd2gkmb_', 'd2gl3a_', 'd2gl3b_', '...",714


In [156]:
labels_specific_to_conditions[1]

array([ 1, -1, -1, -1, -1, -1])

In [189]:
n_true_pos

1

In [158]:
# removing queries that have no true positives
print("#-queries of no true positives: ", results_df[results_df["n_true_pos"]==0].shape[0])
filtered_results_df = results_df[results_df["n_true_pos"]!=0]

#-queries of no true positives:  0


In [159]:
# non-weighted summary result of superfamily/fold level homology ranking
non_weighted_results_df = filtered_results_df.drop(columns=["sid"]).describe().reset_index()
non_weighted_results_df

,index,auroc,auprc,hit1,hit10,n_true_pos,n_true_neg
0,count,0.0,6.0,6.0,6.0,6.000000,6.0
1,mean,NaN,1.0,1.0,1.0,1.666667,0.0
2,std,NaN,0.0,0.0,0.0,1.632993,0.0
3,min,NaN,1.0,1.0,1.0,1.000000,0.0
4,25%,NaN,1.0,1.0,1.0,1.000000,0.0
5,50%,NaN,1.0,1.0,1.0,1.000000,0.0
6,75%,NaN,1.0,1.0,1.0,1.000000,0.0
7,max,NaN,1.0,1.0,1.0,5.000000,0.0


In [160]:

# adding other info with results, and removing seq
final_results_per_queries_df = filtered_results_df.merge(remote_homologs_df, left_on="sid", right_on=seq_id_col, how="left")
final_results_per_queries_df.drop(columns=["seq"], inplace=True)
final_results_per_queries_df.reset_index(drop=True, inplace=True)
final_results_per_queries_df

,sid,auroc,auprc,hit1,hit10,n_true_pos,n_true_neg,span,sccs,cls_label,fold,superfamily,family,description,seq_len
0,d4i0va_,NaN,1.0,1,1,5,0,(A:),a.1.1.1,a,a.1,a.1.1,a.1.1.1,automated matches {Synechococcus sp. [TaxId: 3...,123
1,d1asha_,NaN,1.0,1,1,1,0,(A:),a.1.1.2,a,a.1,a.1.1,a.1.1.2,"Ascaris hemoglobin, domain 1 {Pig roundworm (A...",147
2,d4hswa_,NaN,1.0,1,1,1,0,(A:),a.1.1.2,a,a.1,a.1.1,a.1.1.2,Dehaloperoxidase {Amphitrite ornata [TaxId: 12...,137
3,d1jl7a_,NaN,1.0,1,1,1,0,(A:),a.1.1.2,a,a.1,a.1.1,a.1.1.2,Glycera globin {Marine bloodworm (Glycera dibr...,147
4,d1or4a_,NaN,1.0,1,1,1,0,(A:),a.1.1.2,a,a.1,a.1.1,a.1.1.2,"Heme-based aerotactic transducer HemAT, sensor...",169
5,d3g46a_,NaN,1.0,1,1,1,0,(A:),a.1.1.2,a,a.1,a.1.1,a.1.1.2,Hemoglobin I {Ark clam (Scapharca inaequivalvi...,146


In [161]:
n_queries_per_remhomlevel_df = final_results_per_queries_df[[pos_col, "sid"]].groupby(by=pos_col).count().reset_index()
n_queries_per_remhomlevel_df

,superfamily,sid
0,a.1.1,6


In [190]:
# performance per rem-hom level (superfamily/fold)
final_results_per_remhomlevel_df = final_results_per_queries_df[[pos_col, "auroc", "auprc", "hit1", "hit10", "n_true_pos", "n_true_neg"]].groupby(by=pos_col).mean().reset_index()
final_results_per_remhomlevel_df.sort_values(pos_col, ascending=True, inplace=True)
final_results_per_remhomlevel_df = final_results_per_remhomlevel_df.merge(n_queries_per_remhomlevel_df, on=pos_col)
final_results_per_remhomlevel_df.rename(columns={"n_true_pos":"avg_n_true_pos", "n_true_neg": "avg_n_true_neg", "sid":"n_queries"}, inplace=True)
final_results_per_remhomlevel_df

,superfamily,auroc,auprc,hit1,hit10,avg_n_true_pos,avg_n_true_neg,n_queries
0,a.1.1,NaN,1.0,1.0,1.0,1.666667,0.0,6


In [191]:
# weighted summary result of superfamily/fold level homology ranking
weighted_results_df = final_results_per_remhomlevel_df.drop(columns=[pos_col]).describe().reset_index()
weighted_results_df

,index,auroc,auprc,hit1,hit10,avg_n_true_pos,avg_n_true_neg,n_queries
0,count,0.0,1.0,1.0,1.0,1.000000,1.0,1.0
1,mean,NaN,1.0,1.0,1.0,1.666667,0.0,6.0
2,std,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,min,NaN,1.0,1.0,1.0,1.666667,0.0,6.0
4,25%,NaN,1.0,1.0,1.0,1.666667,0.0,6.0
5,50%,NaN,1.0,1.0,1.0,1.666667,0.0,6.0
6,75%,NaN,1.0,1.0,1.0,1.666667,0.0,6.0
7,max,NaN,1.0,1.0,1.0,1.666667,0.0,6.0


In [192]:
# saving results
final_results_per_queries_df.to_csv(final_results_per_queries_filepath, sep="\t", index=False, header=True)
final_results_per_remhomlevel_df.to_csv(final_results_per_remhom_filepath, sep="\t", index=False, header=True)
weighted_results_df.to_csv(weighted_results_filepath, sep="\t", index=False, header=True)
non_weighted_results_df.to_csv(non_weighted_results_filepath, sep="\t", index=False, header=True)